In [14]:
# %pip install wandb
# %pip install datasets
# %pip install tokenizers

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
import math
import time

In [ ]:
from torch.nn import RMSNorm
from torch.utils.data import DataLoader
from datasets import load_dataset
from dataclasses import dataclass
from tokenizers import Tokenizer
from transformers import AutoTokenizer

 <!-- todo: add tokenizer -->

In [19]:
@dataclass
class modelargs:
    block_size = 512
    batch_size = 32
    embeddings_dim = 512
    attn_dropout = 0.1
    # vocab_size = len(tokenizer.get_vocab())
    ignore_pad_token_in_loss = True
    heads = 8 
    dropout = 0.1
    epochs = 1
    max_lr = 6e-4
    decoder_layers = 6
    weight_decay = 0.1
    beta1 = 0.9
    beta2 = 0.95
    clip=1.0
    experts = 16
    top_experts = 4
    use_shared_experts = True
    base_freq = 100000
    noisy_topk = False
    eps: float = 1e-8
    loss_scale = 0.3 
    useauxFreeLoadBalancingLoss = True
    aux_free_bias_update_rate = 0.001
    mtp_heads = 1
    latent_dim = 64